### Notes

cannot apply to PPG-BP because there are not enough detected peaks (required 3)

In [6]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import scipy
from biosppy.signals import bvp
from scipy import io
from hrvanalysis import get_time_domain_features
from hrvanalysis import get_frequency_domain_features
import h5py
import pandas as pd
from pyampd.ampd import find_peaks

In [7]:
freq = 125
debug = False
dataset_name = 'vitaldb'
filter_name = 'zhang21'
alg_name = 'dagamseh21'
# datapath = '../../../datasets/MIMIC-II/Part1234.mat'
datapath = '../../../datasets/vitaldb/' + dataset_name + '.h5'
feature_interval = 1 # extract every n beats
ppg_idx = 0
ecg_idx = 1
abp_idx = 2

save_name = 'features_' + dataset_name + '_' + filter_name + '_' + alg_name + '.csv'
data = h5py.File(datapath, 'r')
keys = np.array(list(data.keys()))[np.arange(0, len(data.keys()))]
all_features = []

In [8]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

# Extracting BP-related features of each signal
all_features = pd.DataFrame()
for i in range(len(keys)):
    subject_features = pd.DataFrame()
    print("subject = {}".format(i))
    if dataset_name == 'vitaldb':
        seg_idxs = np.arange(0, data[keys[i]].shape[0])
        segs = np.random.choice(seg_idxs, size=np.minimum([10], [data[keys[i]].shape[0]]), replace=False)
    else:
        segs = np.arange(0, data[keys[i]].shape[0])
    for r in range(len(segs)):
#             print(".", end="")
        PPG = data[keys[i]][segs[r], ppg_idx, :]
        BP = data[keys[i]][segs[r], abp_idx, :]
        details = [keys[i].astype(int),segs[r],2,3,np.max(BP), np.min(BP), 6,7,8]
        try:    
            # Filtering the signal        
            # the library uses filtfilt (=> Phase = 0)
            ppg_filtered_low = bvp.st.filter_signal(signal=PPG,ftype='butter',band='lowpass',order=6,frequency=10,sampling_rate=freq)[0]
            if (int(15 * 1000/freq)-1)%2 > 0:
                medfilt_param = int(15 * 1000/freq)-1
            else:
                medfilt_param = int(15 * 1000/freq)
            ppg_filtered = ppg_filtered_low - scipy.signal.medfilt(ppg_filtered_low, medfilt_param)                 
            # Calculating the peaks and minimums of the signal (using AMPD)
            onsets = find_peaks(-1 * ppg_filtered, scale=200)
            peaks = find_peaks(ppg_filtered, scale=200)
            nn_intervals_list  = np.diff(onsets) * (1000/freq)
            time_domain_features = list(get_time_domain_features(nn_intervals_list).values())
            freq_domain_features = list(get_frequency_domain_features(nn_intervals_list).values())

            HR = 60/(np.median(np.diff(onsets))/freq)
            PPG = np.array(PPG)
            record_features = []
            for onset in list(range(1,len(onsets)-1,feature_interval)):
                try:
                    segment_features = []
                    for win_num in range(0,feature_interval):
                        ppg_window = ppg_filtered[onsets[onset + win_num]:onsets[onset + win_num + 1]]
                        ppg_window = ppg_window - np.min(ppg_window)
                        ppg_window = ppg_window / np.max(ppg_window)                          
                        window_features = []
                        ignore_flag = False
                        # Detecting maximum of the window and dividing the Pulse into two ascending and descending parts
                        ppg_window_systolic = np.argmax(ppg_window)
                        ppg_window_ascending = ppg_window[:ppg_window_systolic]
                        ppg_window_descending = ppg_window[ppg_window_systolic:]
                        # Fitting polynomials to each part and calculating their first and second derivatives
                        ppg_window_ascending_x = np.linspace(0,ppg_window_systolic,len(ppg_window_ascending))
                        ppg_window_descending_x = np.linspace(ppg_window_systolic,len(ppg_window),len(ppg_window_descending))
                        asc_weights = np.ones(np.shape(ppg_window_ascending))
                        asc_weights[int(0.25*len(asc_weights)):int(0.75*len(asc_weights))] = 1
                        asc_weights[int(0.4*len(asc_weights)):int(0.6*len(asc_weights))] = 1
                        des_weights = np.ones(np.shape(ppg_window_descending))
                        des_weights[int(0.25*len(des_weights)):int(0.75*len(des_weights))] = 1
                        des_weights[int(0.4*len(des_weights)):int(0.6*len(des_weights))] = 1
                        des_weights[int(0.8*len(des_weights)):int(1*len(des_weights))] = 1
                        ppg_window_ascending_poly5 = np.polyfit(ppg_window_ascending_x ,ppg_window_ascending,5,w=asc_weights)
                        ppg_window_descending_poly7 = np.polyfit(ppg_window_descending_x,ppg_window_descending,7,w=des_weights)
                        ppg_window_ascending_poly5_der = np.poly1d(np.polyder(ppg_window_ascending_poly5))
                        ppg_window_descending_poly7_der = np.poly1d(np.polyder(ppg_window_descending_poly7))
                        ppg_window_descending_poly7_der2 = np.poly1d(np.polyder(ppg_window_descending_poly7,2))
                        ppg_window_ascending_poly5_der_values = ppg_window_ascending_poly5_der(ppg_window_ascending_x)
                        ppg_window_descending_poly7_der_values = ppg_window_descending_poly7_der(ppg_window_descending_x)
                        ppg_window_descending_poly7_der2_values = ppg_window_descending_poly7_der2(ppg_window_descending_x)
                        # calculating the location of Max. slope point
                        ppg_maxslope = np.argmax(ppg_window_ascending_poly5_der_values)
                        # calculating the location of Diastolic peak, Dicrotic Notch and inflection point based on the method presented in our paper
                        ppg_window_descending_poly7_der_roots = np.sort(np.array([np.real(root) for root in np.roots(ppg_window_descending_poly7_der) if ~np.iscomplex(root)]))
                        ppg_window_descending_poly7_der_roots = np.array([root for root in ppg_window_descending_poly7_der_roots if (root>ppg_window_systolic*1.25 and root<0.95*len(ppg_window))])
                        ppg_window_descending_poly7_der_roots_der2_values = ppg_window_descending_poly7_der2(ppg_window_descending_poly7_der_roots)
                        if len(ppg_window_descending_poly7_der_roots)==0:
                            ppg_no_dia = True
                        else:
                            dia_candidates = []
                            for dia_index, dia_candidate in enumerate(ppg_window_descending_poly7_der_roots):
                                if ppg_window_descending_poly7_der_roots_der2_values[dia_index]<0:
                                    dia_candidates.append(dia_candidate)
                            if len(dia_candidates)>0:
                                ppg_dia = dia_candidates[int(len(dia_candidates)/2)]
                                ppg_no_dia = False
                            else:
                                ppg_no_dia = True
                        if ppg_no_dia == True:
                            ppg_des_der2_minimums = scipy.signal.find_peaks(-1*ppg_window_descending_poly7_der2_values)[0] + ppg_window_systolic
                            dia_candidates = np.array([root for root in ppg_des_der2_minimums if (root>ppg_window_systolic*1.25 and root<0.95*len(ppg_window))])
                            if len(dia_candidates)>0:
                                ppg_dia = dia_candidates[0]            
                            else:
                                ppg_dia = ppg_window_systolic + int(len(ppg_window_descending)/2)
                        # find dicrotic notch
                        ppg_des_der2_maximums = scipy.signal.find_peaks(ppg_window_descending_poly7_der2_values)[0] + ppg_window_systolic
                        dicrotic_candidates = np.array([root for root in ppg_des_der2_maximums if (root>ppg_window_systolic*1.2 and root<0.995*ppg_dia)])
                        if len(dicrotic_candidates)>0:
                            ppg_dic = dicrotic_candidates[0]           
                        else:
                            ppg_dic = ppg_des_der2_maximums[0]
                        # find inflection point
                        ppg_window_descending_poly7_der2_roots = np.sort(np.array([np.real(root) for root in np.roots(ppg_window_descending_poly7_der2) if ~np.iscomplex(root)]))
                        inf_candidates = np.array([root for root in ppg_window_descending_poly7_der2_roots if (root>ppg_dic and root<ppg_dia)])
                        if len(inf_candidates)>0:
                            ppg_inf = inf_candidates[0]
                        else:
                            ignore_flag = True
                            ppg_inf = (ppg_dia + ppg_dic)/2

                        ppg_dia = int(ppg_dia)
                        ppg_dic = int(ppg_dic)
                        ppg_inf = int(ppg_inf)

                        # When point detection is done, extract the features
                        window_features.extend(details)
                        meanBP = (1*details[4] + 2*details[5]) / 3
                        window_features.extend([0])                                       
                        ppg_window_main = PPG[onsets[onset + win_num]:onsets[onset + win_num + 1]]
                        ppg_window_main = np.array(ppg_window_main)            
                        ppg_dc = np.mean(PPG)
                        ppg_window_main_rev = ppg_window_main[::-1]
                        try:
                            ppg_last_min = len(ppg_window_main)- scipy.signal.find_peaks(-1*ppg_window_main_rev)[0][0]
                        except:
                            ppg_last_min = len(ppg_window_main) -1
                        ppg_last_min = (ppg_window_main[0]+ppg_window_main[ppg_last_min])/2

                        # features
                        Sa = ppg_window_main[ppg_window_systolic]
                        Da = ppg_window_main[ppg_dia]
                        A1 = np.trapz(ppg_window_main[0:ppg_window_systolic])
                        A2 = np.trapz(ppg_window_main[ppg_window_systolic:ppg_dia])
                        IPA = A1/A2
                        ST = ppg_dic
                        DT = ppg_dia - ppg_dic
                        PI = ppg_window_main[ppg_dia] - ppg_last_min
                        ppg_window_prev = PPG[onsets[onset+win_num-1]:onsets[onset + win_num]] 
                        PPI = len(ppg_window_prev) - scipy.signal.find_peaks(ppg_window_prev)[0][-1] + ppg_window_systolic
                        FWHM = np.mean(scipy.signal.peak_widths(ppg_window_main, scipy.signal.find_peaks(ppg_window_main)[0], rel_height=0.5)[0])
                        AI = Sa/Da
                        SI = Sa/DT
                        window_features.extend([Sa, Da, A1, A2, IPA, ST, DT, PI, PPI, FWHM, AI, SI])
                        segment_features.append(window_features)
                    record_features.append(np.mean(segment_features,0))            
                except:                
#                         print('^', end='')
                    pass
            if len(np.median(record_features,0))>0:
                subject_features = pd.concat([subject_features, pd.DataFrame(np.median(record_features,0)).T.rename({0:r})])
#                     print(subject_features)
#                 print("")
#                 print("++")
        except:                
            print("**")
            pass
    if len(subject_features) > 0:
        all_features = pd.concat([all_features, pd.DataFrame(subject_features).groupby(0).mean().reset_index()])

subject = 0
subject = 1
subject = 2
subject = 3
subject = 4
subject = 5
subject = 6
subject = 7
subject = 8
subject = 9
subject = 10
subject = 11
subject = 12
subject = 13
subject = 14
subject = 15
subject = 16
subject = 17
subject = 18
subject = 19
subject = 20
subject = 21
subject = 22
subject = 23
subject = 24
subject = 25
subject = 26
subject = 27
subject = 28
subject = 29
subject = 30
subject = 31
subject = 32
subject = 33
subject = 34
subject = 35
subject = 36
subject = 37
subject = 38
subject = 39
subject = 40
subject = 41
subject = 42
subject = 43
subject = 44
subject = 45
subject = 46
subject = 47
subject = 48
subject = 49
subject = 50
subject = 51
subject = 52
subject = 53
subject = 54
subject = 55
subject = 56
subject = 57
subject = 58
subject = 59
subject = 60
subject = 61
subject = 62
subject = 63
subject = 64
subject = 65
subject = 66
subject = 67
subject = 68
subject = 69
subject = 70
subject = 71
subject = 72
subject = 73
subject = 74
subject = 75
subject = 76
subject =

subject = 593
subject = 594
subject = 595
subject = 596
subject = 597
subject = 598
subject = 599
subject = 600
subject = 601
subject = 602
subject = 603
subject = 604
subject = 605
subject = 606
subject = 607
subject = 608
subject = 609
subject = 610
subject = 611
subject = 612
subject = 613
subject = 614
subject = 615
subject = 616
subject = 617
subject = 618
subject = 619
subject = 620
subject = 621
subject = 622
subject = 623
subject = 624
subject = 625
subject = 626
subject = 627
subject = 628
subject = 629
subject = 630
subject = 631
subject = 632
subject = 633
subject = 634
subject = 635
subject = 636
subject = 637
subject = 638
subject = 639
subject = 640
subject = 641
subject = 642
subject = 643
subject = 644
subject = 645
subject = 646
subject = 647
subject = 648
subject = 649
subject = 650
subject = 651
subject = 652
subject = 653
subject = 654
subject = 655
subject = 656
subject = 657
subject = 658
subject = 659
subject = 660
subject = 661
subject = 662
subject = 663
subjec

subject = 1167
subject = 1168
subject = 1169
subject = 1170
subject = 1171
subject = 1172
subject = 1173
subject = 1174
subject = 1175
subject = 1176
subject = 1177
subject = 1178
subject = 1179
subject = 1180
subject = 1181
subject = 1182
subject = 1183
subject = 1184
subject = 1185
subject = 1186
subject = 1187
subject = 1188
subject = 1189
subject = 1190
subject = 1191
subject = 1192
subject = 1193
subject = 1194
subject = 1195
subject = 1196
subject = 1197
subject = 1198
subject = 1199
subject = 1200
subject = 1201
subject = 1202
subject = 1203
subject = 1204
subject = 1205
subject = 1206
subject = 1207
subject = 1208
subject = 1209
subject = 1210
subject = 1211
subject = 1212
subject = 1213
subject = 1214
subject = 1215
subject = 1216
subject = 1217
subject = 1218
subject = 1219
subject = 1220
subject = 1221
subject = 1222
subject = 1223
subject = 1224
subject = 1225
subject = 1226
subject = 1227
subject = 1228
subject = 1229
subject = 1230
subject = 1231
subject = 1232
subject = 

subject = 1714
subject = 1715
subject = 1716
subject = 1717
subject = 1718
subject = 1719
subject = 1720
subject = 1721
subject = 1722
subject = 1723
subject = 1724
subject = 1725
subject = 1726
subject = 1727
subject = 1728
subject = 1729
subject = 1730
subject = 1731
subject = 1732
subject = 1733
subject = 1734
subject = 1735
subject = 1736
subject = 1737
subject = 1738
subject = 1739
subject = 1740
subject = 1741
subject = 1742
subject = 1743
subject = 1744
subject = 1745
subject = 1746
subject = 1747
subject = 1748
subject = 1749
subject = 1750
subject = 1751
subject = 1752
subject = 1753
subject = 1754
subject = 1755
subject = 1756
subject = 1757
subject = 1758
subject = 1759
subject = 1760
subject = 1761
subject = 1762
subject = 1763
subject = 1764
subject = 1765
subject = 1766
subject = 1767
subject = 1768
subject = 1769
subject = 1770
subject = 1771
subject = 1772
subject = 1773
subject = 1774
subject = 1775
subject = 1776
subject = 1777
subject = 1778
subject = 1779
subject = 

subject = 2260
subject = 2261
subject = 2262
subject = 2263
subject = 2264
subject = 2265
subject = 2266
subject = 2267
subject = 2268
subject = 2269
subject = 2270
subject = 2271
subject = 2272
subject = 2273
subject = 2274
subject = 2275
subject = 2276
subject = 2277
subject = 2278
subject = 2279
subject = 2280
subject = 2281
subject = 2282
subject = 2283
subject = 2284
subject = 2285
subject = 2286
subject = 2287
subject = 2288
subject = 2289
subject = 2290
subject = 2291
subject = 2292
subject = 2293
subject = 2294
subject = 2295
subject = 2296
subject = 2297
subject = 2298
subject = 2299
subject = 2300
subject = 2301
subject = 2302
subject = 2303
subject = 2304
subject = 2305
subject = 2306
subject = 2307
subject = 2308
subject = 2309
subject = 2310
subject = 2311
subject = 2312
subject = 2313
subject = 2314
subject = 2315
subject = 2316
subject = 2317
subject = 2318
subject = 2319
subject = 2320
subject = 2321
subject = 2322
subject = 2323
subject = 2324
subject = 2325
subject = 

subject = 2806
subject = 2807
subject = 2808
subject = 2809
subject = 2810
subject = 2811
subject = 2812
subject = 2813
subject = 2814
subject = 2815
subject = 2816
subject = 2817
subject = 2818
subject = 2819
subject = 2820
subject = 2821
subject = 2822
subject = 2823
subject = 2824
subject = 2825
subject = 2826
subject = 2827
subject = 2828
subject = 2829
subject = 2830
subject = 2831
subject = 2832
subject = 2833
subject = 2834
subject = 2835
subject = 2836
subject = 2837
subject = 2838
subject = 2839
subject = 2840
subject = 2841
subject = 2842
subject = 2843
subject = 2844
subject = 2845
subject = 2846
subject = 2847
subject = 2848
subject = 2849
subject = 2850
subject = 2851
subject = 2852
subject = 2853
subject = 2854
subject = 2855
subject = 2856
subject = 2857
subject = 2858
subject = 2859
subject = 2860
subject = 2861
subject = 2862
subject = 2863
subject = 2864
subject = 2865
subject = 2866
subject = 2867
subject = 2868
subject = 2869
subject = 2870
subject = 2871
subject = 

subject = 3353
subject = 3354
subject = 3355
subject = 3356


In [11]:
df = pd.DataFrame(all_features)
df = df.drop([0,1,2,3,6,7,8,9], axis=1)
df.replace([np.inf, -np.inf],np.nan, inplace=True)
df.fillna(df.mean(),inplace=True)
df = df.reset_index(drop=True)
# df = df.drop(df[df[4] < 90].index)
# df = df.drop(df[df[5] < 40].index)
# df = df.drop(df[df[10] < 50].index)
# df = df.drop(df[df[4] > 200].index)
# df = df.drop(df[df[5] > 100].index)
# df = df.drop(df[df[10] > 140].index)

col_names_1 = ['SBP','DBP']
# manually extracted features
col_names_2 = ['Sa', 'Da', 'A1', 'A2', 'IPA', 'ST', 'DT', 'PI', 'PPI', 'FWHM', 'AI', 'SI']

col_names = col_names_1.copy()
col_names.extend(col_names_2)
len(col_names)

df.columns = col_names
df.to_csv('../../results/features/' + save_name, index=False)

In [12]:
df

,SBP,DBP,Sa,Da,A1,A2,IPA,ST,DT,PI,PPI,FWHM,AI,SI
0,120.547299,50.339000,62.359276,32.162810,1520.180964,1962.797207,0.862811,66.90,18.10,2.784623,45.00,6.360302,1.967945,2.838533
1,133.877500,58.929881,61.352055,34.967190,1419.845022,1627.797498,0.939581,56.70,18.25,2.656250,68.65,13.397746,1.769694,3.130573
2,113.042551,39.279461,58.587180,31.787585,1967.160011,2128.269644,0.955682,82.70,21.90,2.044018,71.45,13.479786,1.808010,2.758669
3,107.414172,48.067850,65.993094,39.390995,1172.129437,1923.054400,0.751312,51.90,18.30,9.696820,65.40,10.350289,1.726500,1.021902
4,143.851100,99.514331,59.298140,33.091020,1406.474682,1368.229541,1.019228,58.00,11.80,7.514550,67.75,18.795092,1.824629,4.517250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,97.934400,49.746510,62.240771,38.561550,1029.862670,1696.526231,0.815684,52.95,15.70,11.602630,46.05,12.737555,1.692604,0.582190
3353,139.703801,73.544229,58.192190,34.749965,1328.697280,1346.125702,0.991185,52.65,14.10,1.678682,54.95,12.019180,1.683086,3.872673
3354,118.275800,60.312310,60.917570,38.225820,1212.103101,1230.980585,1.209241,43.80,14.15,3.623967,51.90,10.848237,1.607091,6.468621
3355,128.051701,68.211970,55.684045,29.634940,1285.313973,1586.757255,0.878407,59.40,16.30,3.890587,70.35,20.546109,1.928355,3.586382
